In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import wandb
from wandb.keras import WandbCallback
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, preprocessing, optimizers, callbacks

# Function to load data file 
def load_data(file):
    with open(file, 'rb') as fo:
        try:
            samples = pickle.load(fo)
        except UnicodeDecodeError:  # python 3.x
            fo.seek(0)
            samples = pickle.load(fo, encoding='latin1')

    data, labels = samples['data'], samples['labels']

    data = np.array(data, dtype=np.float32) / 255
    labels = np.array(labels, dtype=np.int32)
    return data, labels

In [ ]:
# Load data and combine batches
x_test, y_test = load_data('/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch')

x_train_1, y_train_1 = load_data('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1')
x_train_2, y_train_2 = load_data('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2')
x_train_3, y_train_3 = load_data('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3')
x_train_4, y_train_4 = load_data('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4')
x_train_5, y_train_5 = load_data('/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5')

x_train = np.concatenate((x_train_1, x_train_2, x_train_3, x_train_4, x_train_5))
y_train = np.concatenate((y_train_1, y_train_2, y_train_3, y_train_4, y_train_5))
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Define model 
def make_model(num_ch_c1, num_ch_c2):
    
    model = models.Sequential()
    model.add(layers.Reshape(target_shape=(32, 32, 3), input_shape=(3072,)))
    model.add(layers.Conv2D(num_ch_c1, kernel_size=9, activation='relu', input_shape=(None, None, 3), padding='valid'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(layers.Conv2D(num_ch_c2, kernel_size=5, activation='relu', padding='valid'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(300))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, input_shape=(300,)))
    return model

def main():
    # Fix lucky seed!
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # Hyperparameters
    defaults=dict(
    num_ch_c1 = 90, 
    num_ch_c2 = 40,
    epochs = 1000,
    batch_size = 128,
    learning_rate = 0.001,
    optimizer = 'SGD',
    dropout_layer_1 = '0.5',
    dropout_layer_2 = '0.5',
    )
    
    # Initiate wandb project 
    wandb.init(config=defaults, resume=True, name='Full Dataset', project='NNA2 Test Runs')
    config = wandb.config

    # Call model and define loss function
    model = make_model(config.num_ch_c1, config.num_ch_c2)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Choose optimizer
    if(config.optimizer == 'SGD'):
        SGD = optimizers.SGD(learning_rate=config.learning_rate)
        opt = SGD
    elif(config.optimizer == 'Adam'):
        Adam = optimizers.Adam(learning_rate=config.learning_rate)
        opt = Adam
    else:
        RMSprop = optimizers.RMSprop(learning_rate=config.learning_rate)
        opt = RMSprop

    # Compile and fit data
    model.compile(optimizer=opt, loss=loss, metrics='accuracy')
    model.fit(x_train, y_train,
                batch_size=config.batch_size,
                epochs=config.epochs,
                validation_data=(x_test, y_test),
                callbacks=[WandbCallback()]
                )
  
if __name__ == '__main__':
  main()
  print('Program complete')